<a href="https://colab.research.google.com/github/peculab/AI4JUBO/blob/main/JuboDeath_V5_SMD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install shap plotly xgboost --quiet

In [19]:
!pip uninstall shap -y
!pip install shap --no-deps

Found existing installation: shap 0.48.0
Uninstalling shap-0.48.0:
  Successfully uninstalled shap-0.48.0
  Using cached shap-0.48.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (25 kB)
Using cached shap-0.48.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)


In [20]:
!pip install ace_tools

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from IPython.display import display
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    classification_report, confusion_matrix, mean_absolute_error, r2_score
)

In [22]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [23]:
# read data and put it in a dataframe
# 在 google 工作表載入外部資料 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1MJxCIqPSTQoAnA_Zt9u3FjjvXlWa74pzpC78XDuhQ9o/edit?usp=sharing')
worksheet = gsheets.worksheet("NewTest")  # 指定分頁名稱
worksheet = worksheet.get_all_records()
external = pd.DataFrame(worksheet)
external = external.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
external.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,ADL_first_score,ADL_last_score,ADL_diff,ADL_std,ADL_明顯惡化,意識清醒_max,使用呼吸輔具,活動假牙,使用管路,六個月內住院次數
0,1246,NaN,NaN,NaN,0,180,1,52,0,5,5,5,0,0.000000,0,0.0,1.0,0.0,1.0,2
1,1216,NaN,NaN,NaN,0,180,1,72,0,5,5,5,0,0.000000,0,0.0,1.0,0.0,1.0,2
2,1223,NaN,NaN,NaN,0,180,1,64,0,5,5,5,0,NaN,0,0.0,0.0,0.0,1.0,2
3,1227,NaN,NaN,NaN,0,180,1,70,0,5,0,5,5,1.666667,0,0.0,1.0,0.0,1.0,5
4,1238,NaN,NaN,NaN,0,180,1,48,0,5,0,5,5,2.886751,0,0.0,0.0,0.0,1.0,2


In [24]:
external['呼吸_ADL低'] = ((external['使用呼吸輔具'] == 1) & (external['ADL_總分_max'] < 30)).astype(int)
external['高齡_住院多'] = ((external['預估年齡'] > 85) & (external['六個月內住院次數'] > 2)).astype(int)
external['ADL低_有管路'] = ((external['ADL_總分_max'] < 30) & (external['使用管路'] == 1)).astype(int)
external['DNR_呼吸'] = ((external['DNR_flag'] == 1) & (external['使用呼吸輔具'] == 1)).astype(int)
external['超級重症'] = (
    (external['DNR_flag'] == 1) &
    (external['使用呼吸輔具'] == 1) &
    (external['ADL_總分_max'] < 30) &
    (external['六個月內住院次數'] > 1)
).astype(int)
external["low_adl_no_var"] = ((external["ADL_std"].isnull()) & (external["ADL_總分_max"] < 5)).astype(int)

In [25]:
external.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,423.0,1374.472813,1304.720568,101.0,1188.5,1223.0,1312.000000,20062.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
入家日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
結案日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,423.0,0.404255,0.491328,0.0,0.0,0.0,1.000000,1.000000
觀察天數,423.0,123.595745,70.987948,2.0,43.5,180.0,180.000000,180.000000
性別_is_male,423.0,0.579196,0.494273,0.0,0.0,1.0,1.000000,1.000000
預估年齡,423.0,77.354610,13.391375,21.0,70.0,80.0,87.000000,101.000000
DNR_flag,423.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
ADL_總分_max,423.0,17.706856,23.395206,0.0,0.0,5.0,30.000000,100.000000


In [26]:
ex_missing_info = external.isnull().sum().to_frame(name='Missing Count')
ex_missing_info['Missing Ratio'] = (ex_missing_info['Missing Count'] / len(external)).round(4)
ex_missing_info = ex_missing_info.sort_values(by='Missing Ratio', ascending=True)
ex_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
性別_is_male,0,0.0000
觀察天數,0,0.0000
死亡標記,0,0.0000
預估年齡,0,0.0000
ADL_first_score,0,0.0000
ADL_總分_max,0,0.0000
DNR_flag,0,0.0000
ADL_diff,0,0.0000
ADL_明顯惡化,0,0.0000


In [27]:
# read data and put it in a dataframe
# 在 google 工作表載入訓練資料 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1UFokE2ATcNkejRy4PSCkcbCeYzrxOslhelzpd2OaPlc/edit?usp=sharing')
worksheet = gsheets.worksheet("NewDATA")  # 指定分頁名稱
worksheet = worksheet.get_all_records()
df = pd.DataFrame(worksheet)
df = df.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
df.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,ADL_first_score,ADL_last_score,ADL_diff,ADL_std,ADL_明顯惡化,意識清醒_max,使用呼吸輔具,活動假牙,使用管路,六個月內住院次數
0,1212,NaN,NaN,NaN,0,180,1,58,0,65,50,65,15,3.638034,1,0.0,0.0,0.0,1.0,1
1,1136,NaN,NaN,NaN,0,180,1,67,0,65,50,65,15,6.324555,1,0.0,0.0,0.0,1.0,1
2,1161,NaN,NaN,NaN,0,180,1,53,0,100,100,100,0,0.000000,0,0.0,0.0,0.0,1.0,1
3,1121,NaN,NaN,NaN,0,180,1,51,0,85,60,85,25,8.010410,1,0.0,1.0,0.0,1.0,1
4,1347,NaN,NaN,NaN,0,180,1,70,0,70,45,70,25,14.433757,1,0.0,1.0,0.0,1.0,1


In [28]:
df['呼吸_ADL低'] = ((df['使用呼吸輔具'] == 1) & (df['ADL_總分_max'] < 30)).astype(int)
df['高齡_住院多'] = ((df['預估年齡'] > 85) & (df['六個月內住院次數'] > 2)).astype(int)
df['ADL低_有管路'] = ((df['ADL_總分_max'] < 30) & (df['使用管路'] == 1)).astype(int)
df['DNR_呼吸'] = ((df['DNR_flag'] == 1) & (df['使用呼吸輔具'] == 1)).astype(int)
df['超級重症'] = (
    (df['DNR_flag'] == 1) &
    (df['使用呼吸輔具'] == 1) &
    (df['ADL_總分_max'] < 30) &
    (df['六個月內住院次數'] > 1)
).astype(int)
df["low_adl_no_var"] = ((df["ADL_std"].isnull()) & (df["ADL_總分_max"] < 5)).astype(int)

In [29]:
df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,15172.0,1404.930464,2926.042938,1.0,1157.0,1225.0,1335.250000,100462.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
入家日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
結案日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,15172.0,0.411613,0.492142,0.0,0.0,0.0,1.000000,1.000000
觀察天數,15172.0,135.797060,61.935242,1.0,82.0,180.0,180.000000,180.000000
性別_is_male,15172.0,0.532824,0.498938,0.0,0.0,1.0,1.000000,1.000000
預估年齡,15172.0,78.450633,12.106295,0.0,72.0,80.0,87.000000,125.000000
DNR_flag,15172.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
ADL_總分_max,15172.0,20.465001,29.055388,0.0,0.0,5.0,30.000000,100.000000


In [30]:
df_missing_info = df.isnull().sum().to_frame(name='Missing Count')
df_missing_info['Missing Ratio'] = (df_missing_info['Missing Count'] / len(df)).round(4)
df_missing_info = df_missing_info.sort_values(by='Missing Ratio', ascending=True)
df_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
性別_is_male,0,0.0000
觀察天數,0,0.0000
死亡標記,0,0.0000
預估年齡,0,0.0000
ADL_first_score,0,0.0000
ADL_總分_max,0,0.0000
DNR_flag,0,0.0000
ADL_diff,0,0.0000
ADL_明顯惡化,0,0.0000


📊 如何證明死亡組與對照組的變項分布無顯著差異？

#標準化平均差 (Standardized Mean Difference, SMD) 或 t-test P-value 建議以 SMD 為主（不受樣本數影響）

In [31]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# 確保 '死亡標記' 是二元
df = df[df['死亡標記'].isin([0, 1])].copy()

# 使用 df_missing_info.index 作為變數清單
variables = df_missing_info.index

# 定義計算 SMD 與 P-value 的函數
def compute_stats(df, var, group_col='死亡標記'):
    group0 = df[df[group_col] == 0][var].dropna()
    group1 = df[df[group_col] == 1][var].dropna()

    mean0 = group0.mean()
    mean1 = group1.mean()
    var0 = group0.var(ddof=1)
    var1 = group1.var(ddof=1)

    std_pooled = np.sqrt((var0 + var1) / 2)
    smd = (mean1 - mean0) / std_pooled if std_pooled > 0 else np.nan

    # 計算 P-value
    try:
        stat, pval = ttest_ind(group1, group0, equal_var=False)
    except Exception:
        pval = np.nan

    # NaN 原因說明
    if std_pooled == 0 or np.isnan(smd):
        reason = "兩組皆為常數或無變異"
    elif len(group0.unique()) == 1 and len(group1.unique()) == 1 and group0.unique()[0] == group1.unique()[0]:
        reason = "兩組數值完全一致"
    else:
        reason = ""

    return mean1, mean0, smd, pval, reason

# 計算所有變數的 SMD + P-value
smd_pval_results = []
for var in variables:
    mean1, mean0, smd, pval, reason = compute_stats(df, var)
    smd_pval_results.append({
        "變數": var,
        "死亡組平均": round(mean1, 3) if not np.isnan(mean1) else "NaN",
        "對照組平均": round(mean0, 3) if not np.isnan(mean0) else "NaN",
        "SMD": round(smd, 3) if not np.isnan(smd) else "NaN",
        "P-value": round(pval, 4) if not np.isnan(pval) else "NaN",
        "說明": reason
    })

# 轉為 DataFrame
smd_df = pd.DataFrame(smd_pval_results)

# 分為兩個表：無 NaN 和 含 NaN
smd_df_no_nan = smd_df[smd_df["SMD"] != "NaN"].reset_index(drop=True)
smd_df_with_nan = smd_df[smd_df["SMD"] == "NaN"].reset_index(drop=True)

/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:579: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/usr/local/lib/python3.11/dist-packages/scipy/_lib/deprecation.py:234: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  return f(*args, **kwargs)


In [32]:
smd_df_no_nan

,變數,死亡組平均,對照組平均,SMD,P-value,說明
0,H01_NUM,1394.575,1412.175,-0.006,0.6783,
1,性別_is_male,0.597,0.488,0.221,0.0,
2,觀察天數,72.611,180.0,-3.017,0.0,
3,預估年齡,80.207,77.222,0.25,0.0,
4,ADL_first_score,12.034,12.405,-0.017,0.3072,
5,ADL_總分_max,15.777,23.745,-0.284,0.0,
6,ADL_diff,3.743,11.34,-0.456,0.0,
7,ADL_明顯惡化,0.116,0.325,-0.519,0.0,
8,ADL_last_score,15.777,23.745,-0.284,0.0,
9,呼吸_ADL低,0.247,0.371,-0.271,0.0,


In [33]:
smd_df_with_nan

,變數,死亡組平均,對照組平均,SMD,P-value,說明
0,死亡標記,1.0,0.0,NaN,0.0,兩組皆為常數或無變異
1,DNR_flag,0.0,0.0,NaN,NaN,兩組皆為常數或無變異
2,DNR_呼吸,0.0,0.0,NaN,NaN,兩組皆為常數或無變異
3,超級重症,0.0,0.0,NaN,NaN,兩組皆為常數或無變異
4,活動假牙,0.0,0.0,NaN,NaN,兩組皆為常數或無變異
5,使用管路,1.0,1.0,NaN,NaN,兩組皆為常數或無變異
6,意識清醒_max,0.0,0.0,NaN,NaN,兩組皆為常數或無變異
7,入家日期,NaN,NaN,NaN,NaN,兩組皆為常數或無變異
8,dbname,NaN,NaN,NaN,NaN,兩組皆為常數或無變異
9,結案日期,NaN,NaN,NaN,NaN,兩組皆為常數或無變異


In [34]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, chi2_contingency

# 建立年齡區間
age_bins = [65, 70, 75, 80, 85, 90, np.inf]
age_labels = ['65-69', '70-74', '75-79', '80-84', '85-89', '90+']
df['年齡區間'] = pd.cut(df['預估年齡'], bins=age_bins, labels=age_labels, right=False)

long_table_rows = []

# 修正後的分類變項
categorical_vars = {
    '年齡區間': age_labels,
    '性別_is_male': [0, 1],  # 0=女性, 1=男性
    'DNR_flag': [1],
    '意識清醒_max': [1],
    '使用管路': [1],
    '活動假牙': [1],
    '使用呼吸輔具': [1]
}

continuous_vars = [
    'ADL_總分_max',
    'ADL_first_score',
    '六個月內住院次數'
]

# 處理類別變項
for var, subgroups in categorical_vars.items():
    for val in subgroups:
        sub_df = df[df[var] == val]
        sub_df = sub_df[sub_df['死亡標記'].isin([0, 1])]

        died = sub_df[sub_df['死亡標記'] == 1]
        survived = sub_df[sub_df['死亡標記'] == 0]

        died_n = len(died)
        surv_n = len(survived)
        total_n = died_n + surv_n

        died_pct = died_n / total_n * 100 if total_n > 0 else np.nan
        surv_pct = surv_n / total_n * 100 if total_n > 0 else np.nan

        # 顯示標籤
        if var == '性別_is_male':
            subgroup_label = '男性' if val == 1 else '女性'
            var_label = '性別'
        elif var == 'DNR_flag':
            subgroup_label = '是'
            var_label = 'DNR'
        elif var == '意識清醒_max':
            subgroup_label = '清醒'
            var_label = '意識程度'
        elif var == '使用管路':
            subgroup_label = '使用'
            var_label = '管路'
        elif var == '活動假牙':
            subgroup_label = '使用'
            var_label = '假牙'
        elif var == '使用呼吸輔具':
            subgroup_label = '使用'
            var_label = '呼吸輔具'
        elif var == '年齡區間':
            subgroup_label = val
            var_label = '年齡'
        else:
            subgroup_label = str(val)
            var_label = var

        # 計算 P-value（類別變項使用卡方）
        try:
            ct = pd.crosstab(df[var] == val, df['死亡標記'])
            if ct.shape == (2, 2):
                _, pval, _, _ = chi2_contingency(ct)
                pval = f"{pval:.3f}"
            else:
                pval = "NA"
        except:
            pval = "NA"

        long_table_rows.append({
            '變項': var_label,
            '分類': subgroup_label,
            '死亡組 n': died_n,
            '死亡組 %': f"{died_pct:.1f}%" if not np.isnan(died_pct) else 'NA',
            '存活組 n': surv_n,
            '存活組 %': f"{surv_pct:.1f}%" if not np.isnan(surv_pct) else 'NA',
            'P-value': pval
        })

# 處理連續變項
for var in continuous_vars:
    died = df[df['死亡標記'] == 1][var].dropna()
    survived = df[df['死亡標記'] == 0][var].dropna()

    died_n = len(died)
    surv_n = len(survived)

    died_mean, died_std = died.mean(), died.std()
    surv_mean, surv_std = survived.mean(), survived.std()

    # p-value
    if died_n > 1 and surv_n > 1:
        _, pval = ttest_ind(died, survived, equal_var=False)
        pval = f"{pval:.3f}"
    else:
        pval = "NA"

    long_table_rows.append({
        '變項': var,
        '分類': '',
        '死亡組 n': died_n,
        '死亡組 %': f"{died_mean:.1f} ± {died_std:.1f}",
        '存活組 n': surv_n,
        '存活組 %': f"{surv_mean:.1f} ± {surv_std:.1f}",
        'P-value': pval
    })

# 整理成表格
long_summary_df = pd.DataFrame(long_table_rows)

In [36]:
long_summary_df

,變項,分類,死亡組 n,死亡組 %,存活組 n,存活組 %,P-value
0,年齡,65-69,489,38.5%,780,61.5%,0.050
1,年齡,70-74,641,34.9%,1198,65.1%,0.000
2,年齡,75-79,762,33.7%,1501,66.3%,0.000
3,年齡,80-84,1199,44.8%,1475,55.2%,0.000
4,年齡,85-89,1301,48.9%,1361,51.1%,0.000
5,年齡,90+,1250,47.8%,1363,52.2%,0.000
6,性別,女性,2516,35.5%,4572,64.5%,0.000
7,性別,男性,3729,46.1%,4355,53.9%,0.000
8,DNR,是,0,NA,0,NA,NA
9,意識程度,清醒,0,NA,0,NA,NA


In [35]:
from scipy.stats import ttest_ind

# 為了進行統計檢定與格式整理，我們將針對每個年齡區間與性別群體，
# 分別產生「死亡組」與「非死亡組」的平均、標準差與 p-value。

# 準備欄位
group_cols = ['年齡區間', '性別']
output_rows = []

# 每個變項進行比較
for (age_group, gender), group_df in df.groupby(group_cols):
    row = {'年齡區間': age_group, '性別': gender, '樣本數': len(group_df)}

    # 死亡/非死亡分開
    died = group_df[group_df['死亡標記'] == 1]
    survived = group_df[group_df['死亡標記'] == 0]

    row['死亡數'] = len(died)
    row['死亡率'] = len(died) / len(group_df) if len(group_df) > 0 else np.nan

    for feat in features:
        # 計算平均與標準差
        died_mean = died[feat].mean()
        died_std = died[feat].std()
        surv_mean = survived[feat].mean()
        surv_std = survived[feat].std()

        # t 檢定（排除 NaN）
        t_stat, p_val = ttest_ind(
            died[feat].dropna(), survived[feat].dropna(), equal_var=False
        ) if died[feat].dropna().size > 1 and survived[feat].dropna().size > 1 else (np.nan, np.nan)

        # 加入欄位（格式：mean ± std (p-value)）
        row[f'{feat}_死亡'] = f'{died_mean:.2f} ± {died_std:.2f}' if not np.isnan(died_mean) else 'NA'
        row[f'{feat}_存活'] = f'{surv_mean:.2f} ± {surv_std:.2f}' if not np.isnan(surv_mean) else 'NA'
        row[f'{feat}_p'] = f'{p_val:.3f}' if not np.isnan(p_val) else 'NA'

    output_rows.append(row)

# 整理成 DataFrame
formatted_df = pd.DataFrame(output_rows)

KeyError: '性別'

In [ ]:
formatted_df

In [ ]:
long_summary_df